# Results

In [1]:
from pathlib import Path
import pandas as pd
from src.helpers.metrics import  build_metrics_table

from experiment_config import DegModel,DATA_NAME,ESTIMATION_NAME,PFNET_NAME,N_REP,PRED_STAT

In [2]:
estimation_dir = Path('experiments')/DATA_NAME/ESTIMATION_NAME
pred_dir = estimation_dir/DegModel.name()/'pf_performs'/PFNET_NAME/f"pred_{PRED_STAT}"

## Load Predictions

In [3]:
df_preds = pd.read_csv(pred_dir / f"rul_test_predictions_rep{N_REP}.csv")

In [4]:
pred_dir

PosixPath('experiments/DS06/estimation_thr0.1_q0.01-0.99_filtered_n0.01_s20.0/gamma_flexible_nvcte_onset0.2-0.2/pf_performs/net256x256x128x64x32leaky0.05_emaloss0.9_init0/pred_mean')

## Compute mean prediction

In [5]:
df_preds = df_preds.groupby(["unit", "time"], as_index=False).mean()

## Full-life Predictions

In [6]:
metrics_df = build_metrics_table(df_preds)
metrics_df

,unit,RMSE,Score,PICP,PINAW
0,7,5.717169,51.582123,0.900000,0.334308
1,8,14.319194,180.881027,0.549451,0.250501
2,9,2.294589,12.819882,1.000000,0.350272
3,10,5.146923,49.071039,1.000000,0.284632


In [7]:
metrics_df.drop(columns=["unit"]).mean()

RMSE      6.869469
Score    73.588518
PICP      0.862363
PINAW     0.304928
dtype: float64

## Degradation Predictions

In [8]:
hi_df = pd.read_csv(estimation_dir/'data_test.csv')
test_units = hi_df['unit'].astype(int).unique().tolist()
onsets = {unit: hi_df[(hi_df['unit']==unit) & (hi_df['hs']==0)]['cycle'].values[0] for unit in test_units}
df_degpreds = df_preds[
    df_preds["time"] > df_preds["unit"].map(onsets)
]

In [9]:
metrics_df = build_metrics_table(df_degpreds)
metrics_df

,unit,RMSE,Score,PICP,PINAW
0,7,6.618482,48.357399,0.862069,0.471854
1,8,7.806970,42.807171,0.830508,0.297453
2,9,2.624135,12.487544,1.000000,0.416538
3,10,2.954692,13.874424,1.000000,0.423818


In [10]:
metrics_df.drop(columns=["unit"]).mean()

RMSE      5.001070
Score    29.381634
PICP      0.923144
PINAW     0.402416
dtype: float64

## Benchmark Predictions

In [11]:
df_benchmark = df_preds[df_preds["true_rul"] <= 65]

In [12]:
metrics_df = build_metrics_table(df_benchmark)
metrics_df

,unit,RMSE,Score,PICP,PINAW
0,7,6.232337,49.463837,0.878788,0.411420
1,8,10.189969,72.600459,0.742424,0.288904
2,9,2.329370,12.544745,1.000000,0.361382
3,10,4.305239,30.567160,1.000000,0.341190


In [13]:
metrics_df.drop(columns=["unit"]).mean()

RMSE      5.764229
Score    41.294050
PICP      0.905303
PINAW     0.350724
dtype: float64

| DS01	 | full      | RUL65     | deg       |
|--------|-----------|-----------|-----------|
| RMSE   | 5.068149  | 4.803219  | 4.711211  |
| Score  | 52.206928 | 37.873358 | 28.425670 |
| PICP   | 0.953896  | 0.943182  | 0.920168  |
| PINAW  | 0.312042  | 0.338091  | 0.337272  |

| DS03	 | full      | RUL65     | deg       |
|--------|-----------|-----------|-----------|
| RMSE   | 7.654103  | 7.383116  | 4.767812  |
| Score  | 71.544372 | 57.373969 | 22.033015 |
| PICP   | 0.926416  | 0.909519  | 0.913396  |
| PINAW  | 0.394850  | 0.398448  | 0.403939  |

| DS05   | full      | RUL65     | deg       |
|--------|-----------|-----------|-----------|
| RMSE   | 6.065781  | 5.153894  | 3.125471  |
| Score  | 45.452652 | 28.057284 | 13.593670 |
| PICP   | 0.988020  | 0.984848  | 0.980692  |
| PINAW  | 0.397921  | 0.422600  | 0.448572  |

| DS06	 | full      | RUL65     | deg       |
|--------|-----------|-----------|-----------|
| RMSE   | 6.869469  | 5.764229  | 5.001070  |
| Score  | 73.588518 | 41.294050 | 29.381634 |
| PICP   | 0.862363  | 0.905303  | 0.923144  |
| PINAW  | 0.304928  | 0.350724  | 0.402416  |